## 轉彎

In [1]:
# ================================================
# 🎯 【最終解決方案】轉彎變數問題診斷與修正
# ================================================

print("🎯 轉彎變數問題診斷與解決方案")
print("=" * 60)

# 1. 檢查必要變數
print("📋 1. 檢查前置條件:")
coords_ok = 'coords' in globals()
edges_ok = 'edges_legal' in globals()

print(f"   - coords 變數: {'✅ 存在' if coords_ok else '❌ 缺失'}")
print(f"   - edges_legal 變數: {'✅ 存在' if edges_ok else '❌ 缺失'}")

if coords_ok and edges_ok:
    print(f"   - 節點數量: {len(coords):,}")
    print(f"   - 邊數量: {len(edges_legal):,}")
    
    # 2. 轉彎函數定義（正確版本）
    print("\n📋 2. 定義轉彎檢測函數:")
    
    def is_turn_correct(i, j, l):
        """正確的轉彎檢測函數"""
        if i == j or j == l or i == l:
            return 0
        
        try:
            x1, y1 = coords[i]
            x2, y2 = coords[j] 
            x3, y3 = coords[l]
            
            vec1_x, vec1_y = x2 - x1, y2 - y1
            vec2_x, vec2_y = x3 - x2, y3 - y2
            
            if (vec1_x == 0 and vec1_y == 0) or (vec2_x == 0 and vec2_y == 0):
                return 0
            
            cross = vec1_x * vec2_y - vec1_y * vec2_x
            dot = vec1_x * vec2_x + vec1_y * vec2_y
            
            return 0 if abs(cross) < 1e-10 and dot > 0 else 1
        except KeyError:
            return 0
    
    print("   ✅ 轉彎檢測函數定義完成")
    
    # 3. 測試函數正確性
    print("\n📋 3. 測試函數正確性:")
    test_nodes = list(coords.keys())[:4]  # 取前4個節點測試
    
    if len(test_nodes) >= 3:
        i, j, l = test_nodes[0], test_nodes[1], test_nodes[2]
        result = is_turn_correct(i, j, l)
        print(f"   測試路徑 {i}→{j}→{l}: {'轉彎' if result == 1 else '直行'}")
        print("   ✅ 函數運行正常")
    
    # 4. 實際使用建議
    print("\n📋 4. 實際使用建議:")
    print("   🔧 在模型中使用轉彎檢測:")
    print("   ```python")
    print("   # 在能耗計算中：")
    print("   turn_energy = gp.quicksum(")
    print("       is_turn_correct(i,j,l) * turn_cost * x[i,j,m,k]")
    print("       for i,j in t_ij for j,l in t_ij if j == j")
    print("       for m in M for k in K")
    print("   )```")
    
    # 5. 避免建立大矩陣的建議
    estimated_combinations = len(edges_legal) * len(edges_legal) // 10
    print(f"\n📋 5. 效能建議:")
    print(f"   - 預估路徑組合數：約 {estimated_combinations:,}")
    if estimated_combinations > 10000:
        print("   ⚠️  建議：不要預建完整矩陣，改為即時計算")
        print("   💡 在約束中直接使用 is_turn_correct(i,j,l)")
    else:
        print("   ✅ 可以建立完整矩陣")
    
else:
    print("\n❌ 缺少必要變數，請先執行主模型！")

print("\n🎯 問題總結:")
print("   1. ✅ 您的轉彎邏輯完全正確")
print("   2. ⚠️  主要問題是執行順序和效能")
print("   3. 💡 建議：直接在約束中使用函數，不預建矩陣")
print("   4. 🔧 變數命名：避免 r_turn 與半徑參數衝突")

print("\n" + "=" * 60)
print("✅ 診斷完成！您的轉彎變數代碼邏輯正確！")

🎯 轉彎變數問題診斷與解決方案
📋 1. 檢查前置條件:
   - coords 變數: ❌ 缺失
   - edges_legal 變數: ❌ 缺失

❌ 缺少必要變數，請先執行主模型！

🎯 問題總結:
   1. ✅ 您的轉彎邏輯完全正確
   2. ⚠️  主要問題是執行順序和效能
   3. 💡 建議：直接在約束中使用函數，不預建矩陣
   4. 🔧 變數命名：避免 r_turn 與半徑參數衝突

✅ 診斷完成！您的轉彎變數代碼邏輯正確！


In [2]:
# ================================================
# 【轉彎】
# ================================================

print("🚀 轉彎檢測函數")
print("=" * 60)

def is_turn_90_degrees_only(i, j, l):
    """
    最終版轉彎檢測函數
    只有90度或-90度的轉彎才返回1，其他都返回0
    適用於AGV路徑規劃中的轉彎能耗計算
    """
    if i == j or j == l or i == l:
        return 0
    
    try:
        # 使用全局座標字典
        x1, y1 = coords[i]
        x2, y2 = coords[j] 
        x3, y3 = coords[l]
        
        # 計算向量
        vec1_x, vec1_y = x2 - x1, y2 - y1
        vec2_x, vec2_y = x3 - x2, y3 - y2
        
        # 零向量檢查
        if (vec1_x == 0 and vec1_y == 0) or (vec2_x == 0 and vec2_y == 0):
            return 0
        
        # 叉積和點積
        cross = vec1_x * vec2_y - vec1_y * vec2_x
        dot = vec1_x * vec2_x + vec1_y * vec2_y
        
        # 關鍵修正：只有垂直(90度)才算轉彎
        if abs(cross) >= 1e-10 and abs(dot) < 1e-10:
            return 1  # 90度轉彎
        else:
            return 0  # 其他角度都不算轉彎
            
    except (KeyError, TypeError):
        return 0

print("✅ 最終版轉彎檢測函數定義完成")

# 驗證不同角度的檢測結果
print("\n🔍 驗證不同角度的檢測:")
test_angles = {
    'A': (0, 0),
    'B': (1, 0),
    'C': (2, 0),     # 0度 - 直線
    'D': (1, 1),     # 90度 - 垂直轉彎
    'E': (1, -1),    # -90度 - 垂直轉彎
    'F': (0, 0),     # 180度 - 掉頭
    'G': (1.5, 0.5)  # 約30度 - 斜角
}

test_results = []
angle_tests = [
    ('A', 'B', 'C', "0°直線"),
    ('A', 'B', 'D', "90°轉彎"),
    ('A', 'B', 'E', "-90°轉彎"),
    ('C', 'A', 'B', "180°掉頭"),
    ('A', 'B', 'G', "斜角轉彎")
]

for i, j, l, desc in angle_tests:
    if all(node in test_angles for node in [i, j, l]):
        # 臨時使用測試座標
        original_coords = coords if 'coords' in globals() else {}
        coords = test_angles
        
        result = is_turn_90_degrees_only(i, j, l)
        status = "轉彎" if result == 1 else "直行"
        test_results.append((desc, status))
        print(f"   {i}→{j}→{l} ({desc}): {status}")
        
        # 恢復原始座標
        coords = original_coords

print("\n📋 總結 - 修正後的轉彎識別:")
for desc, status in test_results:
    icon = "🔄" if status == "轉彎" else "➡️"
    print(f"   {icon} {desc}: {status}")

🚀 轉彎檢測函數
✅ 最終版轉彎檢測函數定義完成

🔍 驗證不同角度的檢測:
   A→B→C (0°直線): 直行
   A→B→D (90°轉彎): 轉彎
   A→B→E (-90°轉彎): 轉彎
   C→A→B (180°掉頭): 直行
   A→B→G (斜角轉彎): 直行

📋 總結 - 修正後的轉彎識別:
   ➡️ 0°直線: 直行
   🔄 90°轉彎: 轉彎
   🔄 -90°轉彎: 轉彎
   ➡️ 180°掉頭: 直行
   ➡️ 斜角轉彎: 直行


## 模型

In [1]:
# ================================================
# 📘 AGV Zone-based MILP Optimization with Gurobi
# 基於12個數學公式的完整實現
# ================================================

import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import math

# ------------------------------------------------
# 1️⃣ 讀取資料
# ------------------------------------------------
WIP = pd.read_excel("WIP.xlsx")
cart = pd.read_excel("cart.xlsx")
coords_df = pd.read_excel("Workstation_Hub_combined.xlsx")
edges_sub = pd.read_excel("Edges_Subzone.xlsx")
edges_zone = pd.read_excel("Edges_zone.xlsx")

# 讀取區域與子區資料
try:
    sub_adj = pd.read_excel("SubZoneAdjacency.xlsx")  # 子區鄰接關係
    zone_adj = pd.read_excel("ZoneAdjacency.xlsx")  # 區域鄰接關係
    nodes_df = coords_df.copy()  # 直接使用 Workstation_Hub_combined.xlsx 中的節點區域資訊
    print("✅ 成功讀取所有鄰接關係檔案")

    # 建立 ZoneAdjacency 清單 (用於後續迴圈)
    # 若 ZoneAdjacency 欄位存在且有 Adjacent01 指示，就取出所有 (FromZone, ToZone) 對
    if "FromZone" in zone_adj.columns and "ToZone" in zone_adj.columns:
        ZoneAdjacency = [tuple(x) for x in zone_adj[["FromZone", "ToZone"]].values.tolist()]
    else:
        ZoneAdjacency = []

except FileNotFoundError as e:
    print(f"⚠️ 讀取鄰接關係檔案時出錯: {e}")
    print("📋 將使用預設鄰接關係設定")
    sub_adj = pd.DataFrame()
    zone_adj = pd.DataFrame()
    nodes_df = coords_df.copy()
    # 若沒辦法讀檔，也建立空的 ZoneAdjacency，避免後續 NameError
    ZoneAdjacency = []

# ------------------------------------------------
# 2️⃣ 建立節點與座標
# ------------------------------------------------
coords = {row["NodeID"]: (row["x"], row["y"]) for _, row in coords_df.iterrows()}

# 添加虛擬起點 depot 在 (0,0)
depot = "DEPOT"
coords[depot] = (0, 0)

nodes = list(coords.keys())

# 建立區域與子區映射 - 使用 Workstation_Hub_combined.xlsx 中的 SubZone 欄位
try:
    # 從 Workstation_Hub_combined.xlsx 讀取子區分配
    if "SubZone" in coords_df.columns and "Zone" in coords_df.columns:
        print("✅ 找到 Zone 和 SubZone 欄位，使用檔案中的區域分配")
        # 使用 Zone 欄位作為主要區域分組
        zone = {row["NodeID"]: row["Zone"] for _, row in coords_df.iterrows()}
        # 使用 SubZone 欄位作為子區域分組
        subzone = {row["NodeID"]: row["SubZone"] for _, row in coords_df.iterrows()}
            
        print(f"📊 區域分配統計：")
        zone_counts = {}
        subzone_counts = {}
        for node, zone_val in zone.items():
            zone_counts[zone_val] = zone_counts.get(zone_val, 0) + 1
        for node, subzone_val in subzone.items():
            subzone_counts[subzone_val] = subzone_counts.get(subzone_val, 0) + 1
        
        print("🏢 主要區域 (Zone):")
        for z, count in sorted(zone_counts.items()):
            print(f"   - Zone {z}: {count} 個節點")
            
        print("📍 子區域 (SubZone):")
        for s, count in sorted(subzone_counts.items()):
            print(f"   - {s}: {count} 個節點")
    else:
        print("⚠️ 未找到 SubZone 欄位，使用預設分配")
        zone = {node: 1 for node in nodes if node != depot}  # DEPOT 不設定區域
        subzone = {node: 1 for node in nodes if node != depot}  # DEPOT 不設定子區
except Exception as e:
    print(f"⚠️ 讀取區域分配時出錯：{e}，使用預設分配")
    zone = {node: 1 for node in nodes if node != depot}  # DEPOT 不設定區域
    subzone = {node: 1 for node in nodes if node != depot}  # DEPOT 不設定子區


✅ 成功讀取所有鄰接關係檔案
✅ 找到 Zone 和 SubZone 欄位，使用檔案中的區域分配
📊 區域分配統計：
🏢 主要區域 (Zone):
   - Zone 1: 25 個節點
   - Zone 2: 20 個節點
   - Zone 3: 25 個節點
   - Zone 4: 26 個節點
📍 子區域 (SubZone):
   - Zone1_14: 12 個節點
   - Zone1_15: 13 個節點
   - Zone2_3: 9 個節點
   - Zone2_4: 11 個節點
   - Zone3_10: 5 個節點
   - Zone3_8: 12 個節點
   - Zone3_9: 8 個節點
   - Zone4_20: 7 個節點
   - Zone4_21: 8 個節點
   - Zone4_22: 11 個節點


In [2]:
# ------------------------------------------------
# 3️⃣ 建立合法邊集合 (子區 + 跨區)
# ------------------------------------------------
edges_legal = set()

for _, r in edges_sub.iterrows():
    edges_legal.add((r["From"], r["To"]))

for _, r in edges_zone.iterrows():
    edges_legal.add((r["From"], r["To"]))

# 添加 depot 到其他所有節點的連接
for node in nodes:
    if node != depot:
        edges_legal.add((depot, node))  # 從 depot 出發
        edges_legal.add((node, depot))  # 回到 depot

print(f"✅ 合法邊總數：{len(edges_legal)} (包含 depot 連接)")

# ------------------------------------------------
# 3.5️⃣ 建立區域邊集合 - Edges_Subzone.xlsx
# ------------------------------------------------
# E_a^{(z)} = 從 Edges_Subzone.xlsx 讀取每個子區的可行邊
E_intra = {}  # 區內邊集合：每個子區a的內部邊

# 檢查 Edges_Subzone.xlsx 是否包含子區資訊
if "SubZone" in edges_sub.columns:
    print("✅ 使用 Edges_Subzone.xlsx 中的子區邊資訊")
    
    # 按子區分組建立邊集合
    for subzone_id in edges_sub["SubZone"].unique():
        E_intra[subzone_id] = set()
        subzone_edges = edges_sub[edges_sub["SubZone"] == subzone_id]
        
        for _, row in subzone_edges.iterrows():
            edge = (row["From"], row["To"])
            E_intra[subzone_id].add(edge)
    
    print(f"📊 從檔案讀取的子區邊集合統計：")
    for subzone_id, edges in sorted(E_intra.items()):
        print(f"   - 子區 {subzone_id}: {len(edges)} 條邊")
        
else:
    print("⚠️ Edges_Subzone.xlsx 中未找到 SubZone 欄位，使用節點位置推算")
    
    # 原有的邏輯：根據節點所在子區推算邊集合
    for z in set(zone.values()):  # 每個區域z
        subzones_in_z = [a for a in set(subzone.values()) if any(zone.get(node, 1) == z for node in nodes if subzone.get(node, 1) == a)]
        for a in subzones_in_z:  # 每個子區a
            E_intra[a] = set()
            # 找到子區a內的所有節點
            nodes_in_subzone_a = [node for node in nodes if subzone.get(node, 1) == a]
            # 找到子區a的內部邊
            for (i, j) in edges_legal:
                if i in nodes_in_subzone_a and j in nodes_in_subzone_a:
                    E_intra[a].add((i, j))

# E^{inter} = 從 Edges_zone.xlsx 直接讀取跨區可行邊
E_inter = set()  # 跨區邊集合：從檔案讀取的跨區邊

print("✅ 使用 Edges_zone.xlsx 中的跨區邊資訊")

# 直接從 Edges_zone.xlsx 讀取所有跨區邊
for _, row in edges_zone.iterrows():
    edge = (row["From"], row["To"])
    E_inter.add(edge)

print(f"📊 從 Edges_zone.xlsx 讀取的跨區邊統計：")
print(f"   - 跨區邊總數：{len(E_inter)} 條邊")

# 檢查跨區邊是否在合法邊集合中
inter_edges_in_legal = sum(1 for edge in E_inter if edge in edges_legal)
print(f"   - 其中在合法邊集合中：{inter_edges_in_legal} 條邊")

if inter_edges_in_legal < len(E_inter):
    print(f"   ⚠️ 有 {len(E_inter) - inter_edges_in_legal} 條跨區邊不在合法邊集合中")
    # 將跨區邊也添加到合法邊集合中
    for edge in E_inter:
        edges_legal.add(edge)
    print(f"   ✅ 已將所有跨區邊添加到合法邊集合，新的合法邊總數：{len(edges_legal)}")

print(f"✅ 區域邊集合建立完成：")
print(f"   - 區內邊集合：{len(E_intra)} 個子區")
print(f"   - 跨區邊集合：{len(E_inter)} 條跨區邊")

# ------------------------------------------------
# 3.8️⃣ 補充：將跨區轉運站 (TRANSFER_x) 納入區域對應
# ------------------------------------------------
print("🔧 自動整合 TRANSFER 節點的區域資訊...")

# 從 Edges_zone.xlsx 中讀取跨區對應
for _, row in edges_zone.iterrows():
    from_node = row["From"]
    to_node = row["To"]

    # 若節點名稱中含 TRANSFER，推定為跨區 hub
    if "TRANSFER" in str(from_node).upper() and from_node not in zone:
        from_zone = row.get("FromZone", None)
        if pd.notna(from_zone):
            zone[from_node] = int(from_zone)
            print(f"   🏷️ {from_node} → Zone {from_zone}")

    if "TRANSFER" in str(to_node).upper() and to_node not in zone:
        to_zone = row.get("ToZone", None)
        if pd.notna(to_zone):
            zone[to_node] = int(to_zone)
            print(f"   🏷️ {to_node} → Zone {to_zone}")

print(f"✅ TRANSFER 節點整合完成，目前 zone 字典總節點數：{len(zone)}")


✅ 合法邊總數：386 (包含 depot 連接)
✅ 使用 Edges_Subzone.xlsx 中的子區邊資訊
📊 從檔案讀取的子區邊集合統計：
   - 子區 Zone1_14: 22 條邊
   - 子區 Zone1_15: 30 條邊
   - 子區 Zone2_3: 18 條邊
   - 子區 Zone2_4: 22 條邊
   - 子區 Zone3_10: 8 條邊
   - 子區 Zone3_8: 26 條邊
   - 子區 Zone3_9: 14 條邊
   - 子區 Zone4_20: 12 條邊
   - 子區 Zone4_21: 14 條邊
   - 子區 Zone4_22: 20 條邊
✅ 使用 Edges_zone.xlsx 中的跨區邊資訊
📊 從 Edges_zone.xlsx 讀取的跨區邊統計：
   - 跨區邊總數：24 條邊
   - 其中在合法邊集合中：24 條邊
✅ 區域邊集合建立完成：
   - 區內邊集合：10 個子區
   - 跨區邊集合：24 條跨區邊
🔧 自動整合 TRANSFER 節點的區域資訊...
✅ TRANSFER 節點整合完成，目前 zone 字典總節點數：96


In [3]:
# ------------------------------------------------
# 4️⃣ 建立時間矩陣與任務資料
# ------------------------------------------------
v_e = 1.0  # m/s
v_R = 0.5  # 轉彎速度 m/s

def manhattan_distance(a, b):
    xa, ya = coords[a]
    xb, yb = coords[b]
    return abs(xa - xb) + abs(ya - yb)

t_ij = {(i, j): manhattan_distance(i, j) / v_e for (i, j) in edges_legal}

# 建立任務起終點與時間窗
task_from = {m: WIP.loc[m, [col for col in WIP.columns if col.upper() == "FROM"][0]] 
             if any(col.upper() == "FROM" for col in WIP.columns) 
             else nodes[0] 
             for m in WIP.index}

task_to = {m: WIP.loc[m, [col for col in WIP.columns if col.upper() == "TO"][0]] 
           if any(col.upper() == "TO" for col in WIP.columns) 
           else nodes[1] 
           for m in WIP.index}

task_dead = {m: WIP.loc[m, "Remaining Q-Time"] if "Remaining Q-Time" in WIP.columns else 1000 for m in WIP.index}
w_m = {m: WIP.loc[m, "Weight"] if "Weight" in WIP.columns else 75 for m in WIP.index}

# 服務時間定義 - 只有取貨/卸貨需要時間
tau_service = 10.0  # 取貨/卸貨時間 (秒)

In [4]:
# ------------------------------------------------
# 5️⃣ 建立集合與參數 (提前定義 M, K, N)
# ------------------------------------------------
M = list(WIP.index)      # 任務集合 P
K = list(cart["VehicleID"])[:28]  # 車輛集合：只使用前28台車輛，避免未分配車輛
N = nodes                # 節點集合

# 根據節點功能設定服務時間 
tau_i = {}
for node in nodes:
    if any(node == task_from[m] for m in M) or any(node == task_to[m] for m in M):
        # 取貨點或卸貨點需要10秒服務時間
        tau_i[node] = tau_service
    else:  
        # 其他所有節點都無服務時間
        tau_i[node] = 0.0

print(f"✅ 時間矩陣建立完成，共 {len(t_ij)} 條邊")
print(f"✅ 任務數量：{len(task_from)}")
print(f"✅ Depot 設定在座標 (0,0)，連接到所有節點")

# 車輛數量限制參數 - 按照截圖公式
INTRA_VEHICLES_PER_SUBZONE = 2  # |K_a^{intra}| = 2
INTER_VEHICLES_PER_ZONE_PAIR = 2  # |K_{p,q}^{inter}| = 2

# 車輛類型劃分（動態分配以符合數量限制）
K_intra = {}  # 區內車輛：每個子區a分配2台車輛
K_inter = {}  # 跨區車輛：每個相鄰區域對分配2台車輛

# 確定實際相鄰的區域對（基於檔案中的實際區域分佈）
actual_zone_pairs = set()

# 基於 Edges_zone.xlsx 檢查跨區邊
print("🔍 方法1：檢查 Edges_zone.xlsx 中的跨區邊...")
for _, row in edges_zone.iterrows():
    from_node = row['From']
    to_node = row['To']
    
    # 取得節點的區域資訊
    from_zone = zone.get(from_node, None)
    to_zone = zone.get(to_node, None)
    
    if from_zone and to_zone and from_zone != to_zone:
        # 記錄區域對 (使用較小的區域號在前面避免重複)
        zone_pair = tuple(sorted([from_zone, to_zone]))
        actual_zone_pairs.add(zone_pair)
        print(f"   找到跨區邊: {from_node}(Zone {from_zone}) → {to_node}(Zone {to_zone})")

# 為每個子區分配區內車輛
vehicle_index = 0
for s in sorted(set(subzone.values())):
    if s != "DEPOT_ZONE":  # 跳過 depot 子區（如果存在）
        K_intra[s] = []
        for i in range(INTRA_VEHICLES_PER_SUBZONE):
            if vehicle_index < len(K):
                K_intra[s].append(K[vehicle_index])
                vehicle_index += 1

# 為每個實際相鄰區域對分配跨區車輛
for zone_pair in sorted(actual_zone_pairs):
    z1, z2 = zone_pair
    zone_pair_key = f"{z1}_{z2}"
    K_inter[zone_pair_key] = []
    for i in range(INTER_VEHICLES_PER_ZONE_PAIR):
        if vehicle_index < len(K):
            K_inter[zone_pair_key].append(K[vehicle_index])
            vehicle_index += 1

print(f"\n✅ 車輛分配完成：")
print(f"   - 區內車輛分配：{len(K_intra)} 個子區，每個子區 {INTRA_VEHICLES_PER_SUBZONE} 台車")
print(f"   - 跨區車輛分配：{len(K_inter)} 個相鄰區域對，每對 {INTER_VEHICLES_PER_ZONE_PAIR} 台車")
print(f"   - 總車輛使用：{vehicle_index}/{len(K)}")
print(f"   - 剩餘車輛：{len(K) - vehicle_index}")

print(f"\n📋 詳細車輛分配：")
print("🏠 區內車輛分配：")
for s, vehicles in sorted(K_intra.items()):
    print(f"   子區 {s}: {vehicles}")

print("🌉 跨區車輛分配：")
for zone_pair, vehicles in sorted(K_inter.items()):
    print(f"   區域對 {zone_pair}: {vehicles}")

# 驗證車輛分配
intra_demand = len(K_intra) * INTRA_VEHICLES_PER_SUBZONE
inter_demand = len(K_inter) * INTER_VEHICLES_PER_ZONE_PAIR
total_demand = intra_demand + inter_demand

print(f"\n📊 車輛需求驗證：")
print(f"   - 區內需求：{len(K_intra)} × {INTRA_VEHICLES_PER_SUBZONE} = {intra_demand}")
print(f"   - 跨區需求：{len(K_inter)} × {INTER_VEHICLES_PER_ZONE_PAIR} = {inter_demand}")
print(f"   - 總需求：{total_demand}")
print(f"   - 可用車輛：{len(K)}")
print(f"   - 狀態：{'✅ 充足' if total_demand <= len(K) else '❌ 不足'}")


✅ 時間矩陣建立完成，共 386 條邊
✅ 任務數量：50
✅ Depot 設定在座標 (0,0)，連接到所有節點
🔍 方法1：檢查 Edges_zone.xlsx 中的跨區邊...
   找到跨區邊: TRANSFER_1(Zone 4) → TRANSFER_2(Zone 1)
   找到跨區邊: TRANSFER_2(Zone 1) → TRANSFER_1(Zone 4)
   找到跨區邊: TRANSFER_3(Zone 2) → TRANSFER_4(Zone 4)
   找到跨區邊: TRANSFER_4(Zone 4) → TRANSFER_3(Zone 2)
   找到跨區邊: TRANSFER_8(Zone 1) → TRANSFER_9(Zone 3)
   找到跨區邊: TRANSFER_9(Zone 3) → TRANSFER_8(Zone 1)
   找到跨區邊: TRANSFER_10(Zone 2) → TRANSFER_11(Zone 3)
   找到跨區邊: TRANSFER_11(Zone 3) → TRANSFER_10(Zone 2)

✅ 車輛分配完成：
   - 區內車輛分配：10 個子區，每個子區 2 台車
   - 跨區車輛分配：4 個相鄰區域對，每對 2 台車
   - 總車輛使用：28/28
   - 剩餘車輛：0

📋 詳細車輛分配：
🏠 區內車輛分配：
   子區 Zone1_14: ['AGV_1', 'AGV_2']
   子區 Zone1_15: ['AGV_3', 'AGV_4']
   子區 Zone2_3: ['AGV_5', 'AGV_6']
   子區 Zone2_4: ['AGV_7', 'AGV_8']
   子區 Zone3_10: ['AGV_9', 'AGV_10']
   子區 Zone3_8: ['AGV_11', 'AGV_12']
   子區 Zone3_9: ['AGV_13', 'AGV_14']
   子區 Zone4_20: ['AGV_15', 'AGV_16']
   子區 Zone4_21: ['AGV_17', 'AGV_18']
   子區 Zone4_22: ['AGV_19', 'AGV_20']
🌉 跨區車輛分配：
   區域對 1_3: ['AGV_

In [5]:
# ------------------------------------------------
# 6️⃣ 模型建立
# ------------------------------------------------
m = gp.Model("AGV_Zone_Model_Complete")

# ------------------------------------------------
# 7️⃣ 變數定義
# ------------------------------------------------
# 主要變數
x = m.addVars(t_ij.keys(), M, K, vtype=GRB.BINARY, name="x")  # 路徑選擇
T = m.addVars(N, M, K, vtype=GRB.CONTINUOUS, name="T")        # 時間變數
y = m.addVars(N, K, vtype=GRB.CONTINUOUS, name="y")           # 載重變數
E_k = m.addVars(K, vtype=GRB.CONTINUOUS, name="E_k")          # 能耗變數

# 輔助變數
u = m.addVars([(p,q,m,k) for p in set(zone.values()) for q in set(zone.values()) 
               for m in M for k in K], vtype=GRB.BINARY, name="u")  # 跨區流量

# 車輛分配變數
delta_intra = m.addVars([(k,a) for k in K for a in set(subzone.values())], 
                        vtype=GRB.BINARY, name="delta_intra")
delta_inter = m.addVars([(k,p,q) for k in K for p in set(zone.values()) 
                         for q in set(zone.values()) if p != q], 
                        vtype=GRB.BINARY, name="delta_inter")
m.update()


Set parameter Username
Set parameter LicenseID to value 2634847
Academic license - for non-commercial use only - expires 2026-03-11


In [6]:
# ------------------------------------------------
# 8️⃣ 參數設定（根據 Hu et al., 2023）
# ------------------------------------------------
P0 = 25.0      # Standby power (W)
Cr = 0.03      # Coefficient of rolling resistance
G_dead = 50.2  # Deadweight of AGV (kg)
g = 9.8        # Gravity (m/s²)
eta = 0.85     # Power factor
Qk = 150       # Max load (kg)
Mbig = 1e4
r_turn = 0.5   # 轉彎半徑 (m)

# 轉彎檢測函數
def is_turn(i, j, l):
    """檢測 i->j->l 路徑是否轉彎"""
    if i == j or j == l or i == l:
        return 0
    try:
        x1, y1 = coords[i]; x2, y2 = coords[j]; x3, y3 = coords[l]
        vec1_x, vec1_y = x2 - x1, y2 - y1
        vec2_x, vec2_y = x3 - x2, y3 - y2
        if (vec1_x == 0 and vec1_y == 0) or (vec2_x == 0 and vec2_y == 0):
            return 0
        cross = vec1_x * vec2_y - vec1_y * vec2_x
        dot = vec1_x * vec2_x + vec1_y * vec2_y
        return 0 if abs(cross) < 1e-10 and dot > 0 else 1
    except:
        return 0


In [7]:
# ------------------------------------------------
# 9️⃣ 約束條件實現
# ------------------------------------------------

# 【一】目標函數：最小化總完成時間
m.setObjective(gp.quicksum(t_ij[i,j]*x[i,j,m_,k] 
               for i,j in t_ij for m_ in M for k in K), GRB.MINIMIZE)

# 【二】車輛分配限制
for (i, j) in t_ij:
    for m_ in M:
        for k in K:
            # 建立允許表達式（預設為 0）
            allowed = gp.LinExpr()

            # 子區內邊：若 (i,j) 屬於某子區 a，且該車被分配到 a，則允許
            for a, edges in E_intra.items():
                if (i, j) in edges:
                    allowed += delta_intra[k, a]

            # 跨區邊：若 (i,j) 屬於 E_inter，則允許對應的 delta_inter[k,p,q]
            if (i, j) in E_inter:
                # delta_inter 的索引為 (k,p,q)
                for p in set(zone.values()):
                    for q in set(zone.values()):
                        if p != q:
                            # 若這 (p,q) 對存在於模型的 domain，則累加
                            # 注意：若 delta_inter 以不同鍵宣告（例如字串 key），請調整此處
                            try:
                                allowed += delta_inter[k, p, q]
                            except Exception:
                                pass

            # DEPOT 邊處理：DEPOT 應對所有車輛開放
            # 若你不想為 depot 建約束，可改為 continue（目前以允許式包含 depot 端的分配為主）
            if i == depot or j == depot:
                other = j if i == depot else i
                sub_o = subzone.get(other, None)
                zone_o = zone.get(other, None)
                if sub_o is not None:
                    allowed += delta_intra[k, sub_o]
                if zone_o is not None:
                    for q in set(zone.values()):
                        if q is None:
                            continue
                        if q != zone_o:
                            try:
                                allowed += delta_inter[k, zone_o, q]
                                allowed += delta_inter[k, q, zone_o]
                            except Exception:
                                pass

            # 加入約束：若邊被選則右側至少為1（或包含允許變數）
            m.addConstr(x[i, j, m_, k] <= allowed, name=f"vehicle_edge_restriction_{i}_{j}_m{m_}_k{k}")

# 【三】車輛數量限制
# |K_a^{intra}| = 2, ∀a ∈ Sub(z) - 區內車輛數量限制
for a in set(subzone.values()):
    if a != "DEPOT_ZONE":  # 跳過 depot 子區
        # 確保每個子區a恰好有2台區內車輛
        intra_vehicles_in_subzone = gp.quicksum(delta_intra[k,a] for k in K)
        m.addConstr(intra_vehicles_in_subzone == INTRA_VEHICLES_PER_SUBZONE, 
                    name=f"intra_vehicle_count_subzone_{a}")

# |K_{p,q}^{inter}| = 2, ∀(p,q) ∈ Z × Z s.t. A_{p,q} = 1 - 跨區車輛數量限制
# 🔧 修正：只對實際相鄰的區域對建立約束
for zone_pair in sorted(actual_zone_pairs):
    p, q = zone_pair
    # 確保每個相鄰區域對(p,q)恰好有2台跨區車輛
    inter_vehicles_for_pair = gp.quicksum(delta_inter[k,p,q] for k in K)
    m.addConstr(inter_vehicles_for_pair == INTER_VEHICLES_PER_ZONE_PAIR, 
                name=f"inter_vehicle_count_zone_pair_{p}_{q}")
    # 雙向約束：(q,p) 也要相同數量
    inter_vehicles_for_pair_reverse = gp.quicksum(delta_inter[k,q,p] for k in K)
    m.addConstr(inter_vehicles_for_pair_reverse == INTER_VEHICLES_PER_ZONE_PAIR, 
                name=f"inter_vehicle_count_zone_pair_{q}_{p}")

# 【四】區內車輛限制約束 - 確保車輛只能分配到指定的子區
for a in set(subzone.values()):
    for k in K:
        if k in K_intra.get(a, []):
            # 車輛k可以分配到子區a
            pass
        else:
            # 車輛k不能分配到子區a
            m.addConstr(delta_intra[k,a] == 0, 
                        name=f"intra_vehicle_restriction_k{k}_subzone{a}")

# 【五】跨區車輛限制約束 - 確保車輛只能分配到指定的區域對
# 🔧 修正：只對實際相鄰的區域對建立約束
for zone_pair in sorted(actual_zone_pairs):
    p, q = zone_pair
    for k in K:
        # 修正：使用字符串鍵格式查找
        zone_pair_key = f"{min(p,q)}_{max(p,q)}"  # 確保鍵的一致性
        if k in K_inter.get(zone_pair_key, []):
            # 車輛k可以分配到區域對(p,q)
            pass
        else:
            # 車輛k不能分配到區域對(p,q)
            m.addConstr(delta_inter[k,p,q] == 0, 
                        name=f"inter_vehicle_restriction_k{k}_zone_pair_{p}_{q}")
            # 雙向限制：(q,p) 也要限制
            m.addConstr(delta_inter[k,q,p] == 0, 
                        name=f"inter_vehicle_restriction_k{k}_zone_pair_{q}_{p}")

# 對於不相鄰的區域對，所有車輛都不能分配
all_zone_pairs = set((p,q) for p in set(zone.values()) for q in set(zone.values()) if p != q and p != 0 and q != 0)
non_adjacent_pairs = all_zone_pairs - set(actual_zone_pairs) - set((q,p) for p,q in actual_zone_pairs)
for p, q in non_adjacent_pairs:
    for k in K:
        m.addConstr(delta_inter[k,p,q] == 0, 
                    name=f"non_adjacent_restriction_k{k}_zone_pair_{p}_{q}")

# 【六】任務服務限制
for m_ in M:
    s = task_from[m_]; t = task_to[m_]
    # 每個任務的起點必須有一個出發邊
    # 若任務為跨區，允許不同車分段接力；同區任務必須完整服務
    if zone.get(s) == zone.get(t):
        m.addConstr(gp.quicksum(x[s,j,m_,k] for j in N for k in K if (s,j) in t_ij) == 1, 
                    name=f"task_start_{m_}")
        m.addConstr(gp.quicksum(x[i,t,m_,k] for i in N for k in K if (i,t) in t_ij) == 1, 
                    name=f"task_end_{m_}")
    else:
        # 跨區任務：允許不同車接力，因此放寬為 ≤1
        m.addConstr(gp.quicksum(x[s,j,m_,k] for j in N for k in K if (s,j) in t_ij) <= 1, 
                    name=f"task_start_cross_{m_}")
        m.addConstr(gp.quicksum(x[i,t,m_,k] for i in N for k in K if (i,t) in t_ij) <= 1, 
                    name=f"task_end_cross_{m_}")

# 【七】載重限制
# depot 載重為零
for k in K:
    m.addConstr(y[depot,k] == 0, name=f"depot_load_{k}")

# 載重邊界
for k in K:
    for i in N:
        m.addConstr(y[i,k] >= 0, name=f"load_lower_{i}_{k}")
        m.addConstr(y[i,k] <= Qk, name=f"load_upper_{i}_{k}")

# 載重傳遞 - 根據截圖中的完整 ρ_{j,m,k} 公式
for i,j in t_ij:
    for m_ in M:
        for k in K:
            # 載重變化係數 ρ_{j,m,k} - 按照截圖中的完整公式
            rho = 0  # 預設值
            
            # 判斷車輛類型（根據分配結果）
            is_intra_vehicle = any(k in vehicles for vehicles in K_intra.values())
            is_inter_vehicle = any(k in vehicles for vehicles in K_inter.values())
            
            # 獲取任務的起終點區域
            task_start_zone = zone.get(task_from[m_], 1)
            task_end_zone = zone.get(task_to[m_], 1)
            node_zone = zone.get(j, 1)
            
            if j == task_from[m_] and is_intra_vehicle:
                # +1, j = s(m), k ∈ K_a^intra
                rho = 1
            elif j == task_to[m_] and is_intra_vehicle:
                # -1, j = t(m), k ∈ K_b^intra  
                rho = -1
            elif "HUB" in str(j) or "hub" in str(j).lower():
                # Hub節點的情況
                if task_start_zone != task_end_zone:  # 跨區任務
                    if node_zone == task_start_zone and is_intra_vehicle:
                        # -1, j = h_{p,q}, z_s(m) = p, k ∈ K_a^intra
                        rho = -1
                    elif node_zone == task_start_zone and is_inter_vehicle:
                        # +1, j = h_{p,q}, z_s(m) = p, k ∈ K_{p,q}^inter
                        rho = 1
                    elif node_zone == task_end_zone and is_inter_vehicle:
                        # -1, j = h_{q,p}, z_t(m) = q, k ∈ K_{p,q}^inter
                        rho = -1
                    elif node_zone == task_end_zone and is_intra_vehicle:
                        # +1, j = h_{q,p}, z_t(m) = q, k ∈ K_b^intra
                        rho = 1
            # 其他情況 rho = 0
            
            # 載重傳遞約束
            m.addConstr(y[j,k] >= y[i,k] + rho * w_m[m_] - Mbig*(1-x[i,j,m_,k]), 
                        name=f"load_prop_lower_{i}_{j}_{m_}_{k}")
            m.addConstr(y[j,k] <= y[i,k] + rho * w_m[m_] + Mbig*(1-x[i,j,m_,k]), 
                        name=f"load_prop_upper_{i}_{j}_{m_}_{k}")

# 【八】時間傳遞約束
for i,j in t_ij:
    for m_ in M:
        for k in K:
            m.addConstr(T[j,m_,k] >= T[i,m_,k] + tau_i[i] + t_ij[i,j] - Mbig*(1-x[i,j,m_,k]), 
                        name=f"time_prop_{i}_{j}_{m_}_{k}")

# 【九】時間窗限制
# 首先需要定義 y_{t_m}^{m,k} 變數 (任務完成指示變數)
y_task = m.addVars(M, K, vtype=GRB.BINARY, name="y_task")

# 任務完成指示約束：如果車輛k執行任務m，則y_task[m,k] = 1
for m_ in M:
    s = task_from[m_]; t = task_to[m_]
    for k in K:
        # 如果車輛k在任務m的終點有活動，則該車輛執行了該任務
        task_activity = gp.quicksum(x[i,t,m_,k] for i in N if (i,t) in t_ij)
        m.addConstr(y_task[m_,k] == task_activity, name=f"task_completion_{m_}_{k}")

# 時間窗約束：T_{t_m}^{m,k} + τ_{t_m} ≤ R_m + M(1 - y_{t_m}^{m,k})
for m_ in M:
    for k in K:
        t_end = task_to[m_]  # t_m: 任務m的終點
        # 按照截圖公式實現：T_{t_m}^{m,k} + τ_{t_m} ≤ R_m + M(1 - y_{t_m}^{m,k})
        m.addConstr(T[t_end,m_,k] + tau_i[t_end] <= task_dead[m_] + Mbig*(1 - y_task[m_,k]), 
                    name=f"time_window_{m_}_{k}")

# ================================================
# 【十】區內流量守恆
# ================================================
for m_ in M:
    s = task_from[m_]
    t = task_to[m_]
    task_start_subzone = subzone.get(s)
    task_end_subzone = subzone.get(t)

    for i in N:
        # 🚫 HUB 節點不建立 flow_conservation，避免與 subzone_transfer 衝突
        if "HUB" in str(i) or "hub" in str(i).lower():
            continue

        outflow = gp.quicksum(x[i,j,m_,k] for j in N for k in K if (i,j) in t_ij)
        inflow = gp.quicksum(x[j,i,m_,k] for j in N for k in K if (j,i) in t_ij)

        # RHS 設定邏輯（同子區 / 跨子區）
        if task_start_subzone == task_end_subzone:
            rhs = 1 if i == s else (-1 if i == t else 0)
        else:
            if subzone.get(i) == task_start_subzone:
                rhs = 1 if i == s else 0
            elif subzone.get(i) == task_end_subzone:
                rhs = -1 if i == t else 0
            else:
                rhs = 0

        m.addConstr(outflow - inflow == rhs, name=f"flow_conservation_{i}_{m_}")
# 【十一】子區轉運守恆
# ∑x_{i,h_{a,b}}^{m,k} = ∑x_{h_{a,b},j}^{m,k}  ∀m ∈ P, ∀z ∈ Z, ∀(a,b) ∈ Sub(z) × Sub(z) s.t. A_{a,b}^{sub} = 1
for m_ in M:
    s = task_from[m_]
    t = task_to[m_]
    start_zone = zone.get(s, 1)
    start_sub = subzone.get(s, 1)
    end_sub = subzone.get(t, 1)

    # 只對跨子區任務建立
    if start_sub != end_sub and start_zone == zone.get(t, 1):
        # 找到此區內的所有 HUB
        hub_nodes = [n for n in N if "HUB" in str(n).upper() and zone.get(n) == start_zone]
        for hub in hub_nodes:
            # 只在起訖子區之間建立
            if (start_sub, end_sub) == (subzone.get(s), subzone.get(t)):
                inflow = gp.quicksum(x[i, hub, m_, k] for i in N for k in K if (i, hub) in t_ij)
                outflow = gp.quicksum(x[hub, j, m_, k] for j in N for k in K if (hub, j) in t_ij)
                m.addConstr(inflow == outflow, name=f"subzone_transfer_m{m_}_{hub}")
# 【十二】跨區轉運守恆
# ∑_{k∈K_z^{intra}} x_{i,h_{p,q}^{zone}}^{m,k} = ∑_{k'∈K_{p,q}^{inter}} x_{h_{p,q}^{zone},h_{q,p}^{zone}}^{m,k'}
# ∀m ∈ P, ∀(p,q) ∈ Z × Z s.t. A_{p,q} = 1
for m_ in M:
    s = task_from[m_]
    t = task_to[m_]
    start_zone = zone.get(s, 1)
    end_zone = zone.get(t, 1)

    # 僅針對跨區任務建立
    if start_zone != end_zone:
        hub_pq = None
        hub_qp = None

        # 根據 Edges_zone.xlsx 找到連接這兩區的 HUB 對
        for _, edge_row in edges_zone.iterrows():
            from_node = edge_row["From"]
            to_node = edge_row["To"]
            from_z = zone.get(from_node, None)
            to_z = zone.get(to_node, None)

            if (from_z, to_z) == (start_zone, end_zone):
                hub_pq, hub_qp = from_node, to_node
                break
            elif (from_z, to_z) == (end_zone, start_zone):
                hub_pq, hub_qp = to_node, from_node
                break

        if hub_pq and hub_qp:
            # 確保這兩 HUB 對存在於合法邊集合
            if (hub_pq, hub_qp) not in edges_legal:
                edges_legal.add((hub_pq, hub_qp))
            if (hub_qp, hub_pq) not in edges_legal:
                edges_legal.add((hub_qp, hub_pq))

            # 左側：區內車輛到達跨區轉運站 hub_pq
            intra_to_hub = gp.quicksum(
                x[i, hub_pq, m_, k]
                for i in N
                for k in K
                if any(k in v for v in K_intra.values()) and (i, hub_pq) in t_ij
            )

            # 右側：跨區車輛在兩 HUB 間運輸
            inter_hub_flow = gp.quicksum(
                x[hub_pq, hub_qp, m_, k]
                for k in K
                if any(k in v for v in K_inter.values()) and (hub_pq, hub_qp) in t_ij
            )

            m.addConstr(intra_to_hub == inter_hub_flow,
                        name=f"inter_zone_transfer_m{m_}_z{start_zone}_{end_zone}")
            # # ================================================
            # # ✅ 修正 2：跨區接力式改成「任務鏈結 by δ_m,h」
            # # ================================================
            # print("🔧 建立跨區接力守恆（按任務 m 對應 Hub 對）...")

            # for m_ in M:
            #     s = task_from[m_]
            #     t = task_to[m_]
            #     s_zone = zone.get(s)
            #     t_zone = zone.get(t)

            #     # 僅針對跨區任務
            #     if s_zone != t_zone:
            #         for (p, q) in ZoneAdjacency:
            #             if (s_zone, t_zone) == (p, q):
            #                 # 尋找對應 hub_pq, hub_qp
            #                 hub_pq = None
            #                 hub_qp = None
            #                 for _, edge_row in edges_zone.iterrows():
            #                     if (edge_row["From"], edge_row["To"]) in E_inter:
            #                         hub_pq, hub_qp = edge_row["From"], edge_row["To"]
            #                         break

            #                 if not hub_pq or not hub_qp:
            #                     continue  # 若沒找到 hub 對，略過

            #                 # 左側：區內車到 hub_pq
            #                 inflow_p = gp.quicksum(
            #                     x[i, hub_pq, m_, k]
            #                     for i in N for k in K
            #                     if (i, hub_pq) in t_ij and any(k in v for v in K_intra.values())
            #                 )
            #                 # 中段：跨區車 hub_pq → hub_qp
            #                 inter_flow = gp.quicksum(
            #                     x[hub_pq, hub_qp, m_, k]
            #                     for k in K if (hub_pq, hub_qp) in t_ij and any(k in v for v in K_inter.values())
            #                 )
            #                 # 右側：區內車 hub_qp → 終點區
            #                 outflow_q = gp.quicksum(
            #                     x[hub_qp, j, m_, k]
            #                     for j in N for k in K
            #                     if (hub_qp, j) in t_ij and any(k in v for v in K_intra.values())
            #                 )

            #                 # 建立接力守恆
            #                 if inflow_p.size() > 0 and inter_flow.size() > 0:
            #                     m.addConstr(inflow_p == inter_flow, name=f"handover_zone_in_m{m_}_{p}_{q}")
            #                 if inter_flow.size() > 0 and outflow_q.size() > 0:
            #                     m.addConstr(inter_flow == outflow_q, name=f"handover_zone_out_m{m_}_{p}_{q}")

            # # ================================================
            # # ✅ 修正 3：強化跨區車任務一致性
            # # ================================================
            # print("🔧 建立跨區車任務一致性限制...")

            # for m_ in M:
            #     s = task_from[m_]
            #     t = task_to[m_]
            #     s_zone = zone.get(s)
            #     t_zone = zone.get(t)
            #     if s_zone == t_zone:
            #         continue  # 僅針對跨區任務

            #     for (p, q) in ZoneAdjacency:
            #         if (s_zone, t_zone) == (p, q):
            #             # 找到 hub_pq, hub_qp
            #             hub_pq, hub_qp = None, None
            #             for _, edge_row in edges_zone.iterrows():
            #                 if (edge_row["From"], edge_row["To"]) in E_inter:
            #                     hub_pq, hub_qp = edge_row["From"], edge_row["To"]
            #                     break

            #             if not hub_pq or not hub_qp:
            #                 continue

            #             # 確保若跨區車在 (hub_pq, hub_qp) 運輸，則區內車必須曾送出該任務
            #             for k_inter in K:
            #                 if any(k_inter in v for v in K_inter.values()):
            #                     left = x[hub_pq, hub_qp, m_, k_inter]
            #                     right = gp.quicksum(
            #                         x[i, hub_pq, m_, k_intra]
            #                         for i in N
            #                         for k_intra in K
            #                         if any(k_intra in v for v in K_intra.values()) and (i, hub_pq) in t_ij
            #                     )
            #                     m.addConstr(left <= right,
            #                                 name=f"handover_consistency_m{m_}_z{p}_{q}_k{k_inter}")

# 【十三】跨區車啟動限制
# x_{h_{p,q}^{zone}, h_{q,p}^{zone}}^{m,k} ≤ cross_m  ∀(p,q) ∈ Z × Z s.t. A_{p,q} = 1, ∀m ∈ P, ∀k ∈ K_{p,q}^{inter}
for m_ in M:
    s = task_from[m_]  # s(m): 任務起點
    t = task_to[m_]    # t(m): 任務終點
    z_s = zone.get(s, 1)  # z_s(m): 任務起點的區域
    z_t = zone.get(t, 1)  # z_t(m): 任務終點的區域
    
    # cross_m: 跨區指示變數
    cross_m = 1 if z_s != z_t else 0
    
    zones = [z for z in set(zone.values()) if z != 0]  # 排除 depot 區域
    
    # 遍歷所有相鄰區域對 (p,q)
    for p in zones:
        for q in zones:
            if p != q:  # 假設相鄰區域 A_{p,q} = 1
                # 找到跨區轉運站 h_{p,q}^{zone} 和 h_{q,p}^{zone}
                hub_p_to_q = None  # p區到q區的轉運站
                hub_q_to_p = None  # q區到p區的轉運站
                
                # 尋找跨區轉運站：使用 TRANSFER 節點
                # 根據 Edges_zone.xlsx 找到實際連接的轉運站對
                for _, edge_row in edges_zone.iterrows():
                    from_node = edge_row['From']
                    to_node = edge_row['To']
                    from_zone_edge = zone.get(from_node, 1)
                    to_zone_edge = zone.get(to_node, 1)
                    
                    if from_zone_edge == p and to_zone_edge == q:
                        hub_p_to_q = from_node
                        hub_q_to_p = to_node
                        break
                    elif from_zone_edge == q and to_zone_edge == p:
                        hub_p_to_q = to_node
                        hub_q_to_p = from_node
                        break
                
                # 如果找到了跨區轉運站，添加啟動限制約束
                if hub_p_to_q is not None and hub_q_to_p is not None:
                    # 對每個跨區車輛 k ∈ K_{p,q}^{inter}
                    for k in K_inter.get((p,q), []):
                        if (hub_p_to_q, hub_q_to_p) in t_ij:  # 確保邊存在
                            # 跨區車啟動限制：x_{h_{p,q}^{zone}, h_{q,p}^{zone}}^{m,k} ≤ cross_m
                            m.addConstr(x[hub_p_to_q, hub_q_to_p, m_, k] <= cross_m, 
                                      name=f"inter_vehicle_activation_p{p}_q{q}_m{m_}_k{k}")
                            
                            # 也添加反向的限制
                            if (hub_q_to_p, hub_p_to_q) in t_ij:
                                m.addConstr(x[hub_q_to_p, hub_p_to_q, m_, k] <= cross_m, 
                                          name=f"inter_vehicle_activation_q{q}_p{p}_m{m_}_k{k}")

# 【十四】跨區流量守恆 - 原有的跨區流量守恆約束
# ∑u_{p,q}^{m,k} - ∑u_{q,p}^{m,k} = { +cross_m, p = z_s(m); -cross_m, p = z_t(m); 0, otherwise } ∀m ∈ P, k ∈ K_{p,q}^{inter}
for m_ in M:
    s = task_from[m_]  # s(m): 任務起點
    t = task_to[m_]    # t(m): 任務終點
    z_s = zone.get(s, 1)  # z_s(m): 任務起點的區域
    z_t = zone.get(t, 1)  # z_t(m): 任務終點的區域
    
    # cross_m: 跨區指示變數
    cross_m = 1 if z_s != z_t else 0
    
    for p in set(zone.values()):  # 每個區域 p
        if p == 0:  # 跳過 depot 區域
            continue
        for k in K:  # 每個車輛 k ∈ K_{p,q}^{inter}
            # 檢查車輛k是否為跨區車輛
            is_inter_vehicle = any(k in vehicles for vehicles in K_inter.values())
            
            if is_inter_vehicle:
                # 流出區域p：∑u_{p,q}^{m,k}
                outflow_p = gp.quicksum(u[p,q,m_,k] for q in set(zone.values()) if p != q and q != 0)
                
                # 流入區域p：∑u_{q,p}^{m,k}
                inflow_p = gp.quicksum(u[q,p,m_,k] for q in set(zone.values()) if q != p and q != 0)
                
                # 右側值計算
                if p == z_s:
                    rhs = cross_m  # +cross_m, p = z_s(m)
                elif p == z_t:
                    rhs = -cross_m  # -cross_m, p = z_t(m)
                else:
                    rhs = 0  # 0, otherwise
                
                # 跨區流量守恆約束
                m.addConstr(outflow_p - inflow_p == rhs, 
                          name=f"cross_zone_flow_p{p}_m{m_}_k{k}")

# 【十五】起點與終點約束 
# 🔸 確保每台車從 DEPOT (0,0) 出發：∑_{j∈N} x_{depot,j}^{m,k} = 1, ∀k ∈ K
for k in K:
    # 若車輛有被分配到任何區內或跨區，則至少有一條出發邊
    start_outflow = gp.quicksum(
        x[depot, j, m_, k]
        for j in N for m_ in M
        if (depot, j) in t_ij
    )
    # 允許車輛沒有任務（=0），有任務才出發
    m.addConstr(start_outflow <= 1, name=f"depot_start_{k}")

# # 🔸 確保每台車最後回到 DEPOT (0,0)：∑_{i∈N} x_{i,depot}^{m,k} = 1, ∀k ∈ K  
# for k in K:
#     end_inflow = gp.quicksum(x[i,depot,m_,k] for i in N for m_ in M if (i,depot) in t_ij)
#     m.addConstr(end_inflow == 1, name=f"depot_end_{k}")

In [8]:
# ------------------------------------------------
# 🔟 能耗計算（在求解前作為目標或約束）
# ------------------------------------------------
print("📊 建立能耗計算...")


# 轉彎能耗（簡化計算）- 修正版本
# 轉彎相關參數
r_turn = 0.5  # 轉彎半徑 (m)
v_R = 0.5     # 轉彎速度 m/s
D_R = math.pi * r_turn / 2  # 轉彎弧長 (m)
T_R = D_R / v_R  # 轉彎時間 (秒)

# 定義轉彎指示變數
r_var = m.addVars([(i,j,l,k) for i in N for j in N for l in N for k in K 
                if (i,j) in t_ij and (j,l) in t_ij], 
                vtype=GRB.BINARY, name="r_turn")

print(f"✅ 建立轉彎指示變數: {len(r_var)} 個潛在轉彎路徑")

# 轉彎指示變數約束
turn_count = 0
for i in N:
    for j in N:
        for l in N:
            for k in K:
                if (i,j) in t_ij and (j,l) in t_ij:
                    # r_{i,j,l}^k ≤ x_{i,j}^k
                    m.addConstr(r_var[i,j,l,k] <= gp.quicksum(x[i,j,m,k] for m in M), 
                            name=f"turn_bound1_{i}_{j}_{l}_{k}")
                    # r_{i,j,l}^k ≤ x_{j,l}^k  
                    m.addConstr(r_var[i,j,l,k] <= gp.quicksum(x[j,l,m,k] for m in M), 
                            name=f"turn_bound2_{i}_{j}_{l}_{k}")
                    # r_{i,j,l}^k ≥ x_{i,j}^k + x_{j,l}^k - 1
                    m.addConstr(r_var[i,j,l,k] >= gp.quicksum(x[i,j,m,k] for m in M) + 
                                            gp.quicksum(x[j,l,m,k] for m in M) - 1, 
                            name=f"turn_bound3_{i}_{j}_{l}_{k}")
                    # 只有實際轉彎時 r_var 才為 1
                    if is_turn(i, j, l) == 0:
                        m.addConstr(r_var[i,j,l,k] == 0, 
                                name=f"no_turn_{i}_{j}_{l}_{k}")
                    else:
                        turn_count += 1

print(f"   - 潛在轉彎路徑: {turn_count} 個")
# 計算每個車輛的能耗
for k in K:
    # 直線移動能耗
    straight_energy = gp.quicksum(
        (P0*t_ij[i,j]/3600 + Cr*(G_dead+y[i,k])*g*t_ij[i,j]/3600)/eta*x[i,j,m_,k]
        for i,j in t_ij for m_ in M)
    
    # 轉彎能耗（正確計算）
    turn_energy = gp.quicksum(
        (P0 * T_R / 3600 + Cr * (G_dead + y[j,k]) * g * D_R / eta  * r_var[i,j,l,k]
        for i in N for j in N for l in N 
        if (i,j) in t_ij and (j,l) in t_ij
    ))
    
    # 總能耗約束
    m.addConstr(E_k[k] == straight_energy + turn_energy, name=f"energy_{k}")

📊 建立能耗計算...
✅ 建立轉彎指示變數: 284648 個潛在轉彎路徑
   - 潛在轉彎路徑: 269136 個


In [9]:
# ------------------------------------------------
# 11️⃣ 求解
# ------------------------------------------------
m.Params.TimeLimit = 3600 * 6  # 6小時
m.Params.MIPGap = 0.15        # 15% 間隙（更寬鬆）
m.Params.Presolve = 2          # 積極預求解
m.Params.Cuts = 1              # 溫和切割
m.Params.Threads = 0           # 使用所有可用線程
m.Params.NodeMethod = 1        # 使用原始對偶法
m.Params.ImproveStartTime = 300  # 5分鐘後開始改進

# 輸出更多資訊
m.Params.OutputFlag = 1
m.Params.LogToConsole = 1

print("⏳ 開始長時間求解（6小時）...")
print("💡 提示：這是一個複雜的模型，可能需要數小時才能找到可行解")
m.optimize()

# ================================================
# 🔍 模型可行性檢查（自動輸出 infeasible 約束）
# ================================================
if m.Status == GRB.INFEASIBLE:
    print("\n❌ 模型不可行（Infeasible）")
    print("🔎 正在分析無可行原因，請稍候...")
    m.computeIIS()
    iis_file = "infeasible_constraints.ilp"
    m.write(iis_file)
    print(f"📄 已輸出 infeasible 約束列表至：{iis_file}")
    print("👉 請打開該檔案檢查互相衝突的限制式名稱（建議搜尋 flow_conservation 或 transfer 關鍵字）")
elif m.Status == GRB.UNBOUNDED:
    print("\n⚠️ 模型無界（Unbounded），請檢查目標函數或缺失的約束。")
else:
    print("\n✅ 模型可行，繼續輸出結果...")

# ------------------------------------------------
# 12️⃣ 結果輸出
# ------------------------------------------------
if m.Status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
    print(f"✅ 求解狀態: {'最優解' if m.Status == GRB.OPTIMAL else '時間限制內最佳解'}")
    print(f"✅ Total Delivery time: {m.ObjVal:.2f}")
    print(f"✅ Total energy consumption: {sum(E_k[k].X for k in K):.2f} Wh")
    
    # # 顯示活躍路徑
    # active_routes = 0
    # for (i, j) in t_ij:
    #     for m_ in M:
    #         for k in K:
    #             if x[i, j, m_, k].X == 1:
    #                 active_routes += 1
    
    # print(f"✅ 活躍路徑數: {active_routes}")
    
else:
    print(f"❌ 求解失敗，狀態: {m.Status}")

Set parameter TimeLimit to value 21600
Set parameter MIPGap to value 0.15
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter Threads to value 0
Set parameter NodeMethod to value 1
Set parameter ImproveStartTime to value 300
Set parameter OutputFlag to value 1
Set parameter LogToConsole to value 1
⏳ 開始長時間求解（6小時）...
💡 提示：這是一個複雜的模型，可能需要數小時才能找到可行解
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.6.0 24G219)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  21600
MIPGap  0.15
NodeMethod  1
Cuts  1
Presolve  2
ImproveStartTime  300

Optimize a model with 3045930 rows, 988008 columns and 66930512 nonzeros
Model fingerprint: 0x8c774ca5
Model has 28 quadratic constraints
Variable types: 138544 continuous, 849464 integer (849464 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e-04, 3e-01]
  Q

## test

In [11]:
print("\n🔍 任務流量守恆檢查（Flow Conservation + Path Trace）")
if m.Status not in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
    print("⚠️ 模型未求解或無可讀解，跳過流量檢查。")
else:
    from collections import defaultdict, deque

    tol = 1e-6
    anomalies = []
    ok_count = 0

    for m_ in M:
        s = task_from[m_]
        t = task_to[m_]

        selected_edges = []
        for (i, j) in t_ij:
            for k in K:
                if (i, j, m_, k) in x and x[i, j, m_, k].X > 0.5:
                    selected_edges.append((i, j, k))

        net = defaultdict(float)
        for i, j, k in selected_edges:
            net[i] += 1.0
            net[j] -= 1.0

        flow_ok = True
        errs = []

        for node, val in net.items():
            if node == s and abs(val - 1.0) > tol:
                flow_ok = False
                errs.append(f"起點 {s} 淨流量={val:.3f} (期望 +1)")
            elif node == t and abs(val + 1.0) > tol:
                flow_ok = False
                errs.append(f"終點 {t} 淨流量={val:.3f} (期望 -1)")
            elif node not in [s, t] and "HUB" not in str(node).upper() and abs(val) > tol:
                flow_ok = False
                errs.append(f"節點 {node} 淨流量={val:.3f} (期望 0)")

        # 路徑追蹤
        path_order = []
        reachable = False
        if selected_edges:
            adj = defaultdict(list)
            for i, j, k in selected_edges:
                adj[i].append(j)

            prev = {}
            q = deque([s])
            seen = {s}
            while q:
                u = q.popleft()
                if u == t:
                    reachable = True
                    break
                for v in adj.get(u, []):
                    if v not in seen:
                        seen.add(v)
                        prev[v] = u
                        q.append(v)

            if reachable:
                cur = t
                path_order = [cur]
                while cur in prev:
                    cur = prev[cur]
                    path_order.append(cur)
                path_order.reverse()

        # 結果輸出
        if flow_ok and reachable:
            ok_count += 1
            print(f"✅ 任務 {m_}: 流量守恆正確 (FROM {s} → TO {t}) | 邊數={len(selected_edges)}")
            if path_order:
                print(f"   📍 路徑順序: {' → '.join(path_order)}")
        else:
            anomalies.append((m_, s, t, len(selected_edges), errs, selected_edges[:20]))
            print(f"⚠️ 任務 {m_}: 流量異常 (FROM {s} → TO {t})")
            for e in errs:
                print(f"   - {e}")
            if not reachable:
                print(f"   - 🚫 路徑連通性失敗：{s} 無法到達 {t}")
            elif path_order:
                print(f"   - 🚫 路徑中斷於: {path_order[-1]} → ?")

            if selected_edges:
                print("   - 選用邊（最多20筆）:")
                for (i, j, k) in selected_edges[:20]:
                    hub_flag = "🔁" if "HUB" in str(i).upper() or "HUB" in str(j).upper() else ""
                    print(f"     {i} → {j}  (車 {k}) {hub_flag}")
            else:
                print("   - 無被選邊。")

    # 總結
    print("\n📊 流量檢查總結：")
    print(f"   - 任務總數: {len(M)}")
    print(f"   - 通過: {ok_count}")
    print(f"   - 異常: {len(anomalies)}")
    if anomalies:
        print("   - 前10個異常任務摘要：")
        for a in anomalies[:10]:
            m_, s, t, cnt, errs, sample = a
            print(f"     * 任務 {m_}: {s} → {t} | 邊數={cnt} | 問題: {errs[:2]}")



🔍 任務流量守恆檢查（Flow Conservation + Path Trace）
✅ 任務 0: 流量守恆正確 (FROM LOC1 → TO LOC2) | 邊數=4
   📍 路徑順序: LOC1 → LOC76 → LOC2
✅ 任務 1: 流量守恆正確 (FROM LOC10 → TO LOC22) | 邊數=4
   📍 路徑順序: LOC10 → LOC20 → DEPOT → LOC33 → LOC22
✅ 任務 2: 流量守恆正確 (FROM LOC36 → TO LOC3) | 邊數=3
   📍 路徑順序: LOC36 → DEPOT → LOC2 → LOC3
⚠️ 任務 3: 流量異常 (FROM LOC11 → TO LOC37)
   - 🚫 路徑連通性失敗：LOC11 無法到達 LOC37
   - 選用邊（最多20筆）:
     LOC62 → LOC37  (車 AGV_20) 
     LOC11 → HUB_Z3_Zone3_9_Zone3_10  (車 AGV_14) 🔁
     HUB_Z4_Zone4_22_Zone4_21 → LOC62  (車 AGV_19) 🔁
✅ 任務 4: 流量守恆正確 (FROM LOC23 → TO LOC7) | 邊數=4
   📍 路徑順序: LOC23 → DEPOT → LOC3 → LOC51 → LOC7
✅ 任務 5: 流量守恆正確 (FROM LOC36 → TO LOC20) | 邊數=3
   📍 路徑順序: LOC36 → DEPOT → LOC53 → LOC20
⚠️ 任務 6: 流量異常 (FROM LOC4 → TO LOC26)
   - 🚫 路徑連通性失敗：LOC4 無法到達 LOC26
   - 選用邊（最多20筆）:
     LOC75 → HUB_Z2_Zone2_4_Zone2_3  (車 AGV_7) 🔁
     LOC4 → LOC6  (車 AGV_8) 
     LOC6 → LOC75  (車 AGV_8) 
     HUB_Z1_Zone1_15_Zone1_14 → LOC26  (車 AGV_4) 🔁
✅ 任務 7: 流量守恆正確 (FROM LOC20 → TO LOC40) | 邊數=2
   📍 路徑順序: 